# Model Processing

## Imports & General Settings 

In [20]:
import unittest
import os
import sys
import time
import pathlib

import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToPILImage
import PIL
from tqdm.notebook import trange, tqdm
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, roc_curve, roc_auc_score

from matplotlib import pyplot

# Our imports
from data import WaveletTransform, AFECGDataset, SecondDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train, test
import utils


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [22]:
dataset_name = 'afdb'
dataset2 = SecondDataset(dataset_name, '../data/files/')
dataset2.load('./data')

Loaded 2300 samples from backup
torch.Size([2300])
torch.Size([2300, 20, 375])


In [23]:
example, label = dataset2[0]
example

tensor([[0.5568, 0.5416, 0.5116,  ..., 0.0055, 0.0058, 0.0059],
        [0.5509, 0.5371, 0.5101,  ..., 0.0057, 0.0060, 0.0061],
        [0.5441, 0.5317, 0.5073,  ..., 0.0059, 0.0062, 0.0064],
        ...,
        [0.6811, 0.6756, 0.6658,  ..., 0.0035, 0.0045, 0.0055],
        [0.8220, 0.8225, 0.8183,  ..., 0.0065, 0.0072, 0.0079],
        [0.9911, 0.9980, 1.0000,  ..., 0.0162, 0.0161, 0.0160]],
       dtype=torch.float64)

In [ ]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [ ]:
dataset.load()

In [ ]:
data_af, label_af = dataset[0]
data_nsr, label_nsr = dataset[1]

t = data_nsr[0]
utils.show_spectrogram(t)

In [ ]:
images_per_sample = 20
total_data_size = len(dataset)
print("Total data size: ", total_data_size)
print("Patients with AF: ", dataset.labels.sum().item())

In [ ]:
# data = [dataset[i][0] for i in range(total_data_size)]
# labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [ ]:
# samples, label = data[0], labels[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [ ]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# t = to_wavelet(data[0][0])
# image_test = (t * 100 * 255).int() # Simple visualization test
# transforms.ToPILImage()(image_test).show()

##  Wavelet Transform

In [ ]:
# Total data size is 1397
# You can choose the data size 
data_size = len(dataset)

In [ ]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet((375, 20))

# display(x0.unsqueeze(0).shape)
# h = encoder_cnn(x0.unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

## BRNN

In [ ]:
display(BRNN(50, 50, images_per_sample))

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [ ]:
SoftmaxAttention(100)

## Training

In [ ]:
model = Baseline()

### CNN

In [ ]:
data1, label1 = dataset[0]
utils.show_spectrogram(data1[0])

In [28]:
heldout = int(len(dataset2) * 0.2)
train_dataset2, test_dataset2 = torch.utils.data.random_split(dataset2, [len(dataset2) - heldout, heldout])

model = nn.Sequential(
    ConvNet(size=(375, 20), batch=False),
    nn.Linear(50, 2)
)
config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    
    num_epochs=200,

#     num_epochs=200,
    is_notebook=True
)

train(model, train_dataset2, config)

Batch:  torch.Size([90, 20, 375])
Loss:  tensor(0.6842, grad_fn=<NllLossBackward>)
Output: tensor([[-0.0801, -0.0773],
        [-0.0654, -0.1028],
        [-0.0593, -0.0967],
        [-0.0668, -0.0968],
        [-0.0805, -0.0898],
        [-0.0636, -0.0997],
        [-0.0599, -0.0793],
        [-0.0739, -0.0872],
        [-0.0683, -0.0959],
        [-0.0677, -0.1104],
        [-0.0891, -0.0909],
        [-0.0662, -0.1041],
        [-0.0678, -0.0966],
        [-0.0733, -0.1016],
        [-0.0680, -0.0848],
        [-0.0864, -0.0704],
        [-0.0674, -0.0961],
        [-0.0651, -0.0894],
        [-0.0754, -0.0790],
        [-0.0652, -0.0960],
        [-0.0670, -0.0928],
        [-0.0724, -0.0794],
        [-0.0785, -0.0740],
        [-0.0486, -0.1055],
        [-0.0660, -0.0996],
        [-0.0557, -0.1008],
        [-0.0686, -0.0975],
        [-0.0980, -0.0840],
        [-0.0852, -0.0678],
        [-0.0673, -0.0987],
        [-0.0721, -0.1035],
        [-0.0732, -0.0963],
        [-0.0

Output: tensor([[ 4.7499, -3.9507],
        [ 4.0517, -3.4035],
        [ 4.1695, -3.4938],
        [ 3.5567, -2.9902],
        [ 4.9019, -4.0829],
        [ 4.4128, -3.6907],
        [ 4.1672, -3.4965],
        [ 4.7104, -3.9255],
        [ 4.4058, -3.6888],
        [ 4.6935, -3.9073],
        [ 4.0742, -3.4384],
        [ 4.8533, -4.0303],
        [ 4.1337, -3.4694],
        [ 4.0322, -3.3936],
        [ 4.6808, -3.8924],
        [ 3.8271, -3.2441],
        [ 4.3928, -3.6691],
        [ 4.9087, -4.0753],
        [ 4.2519, -3.5732],
        [ 4.7570, -3.9768],
        [ 4.2447, -3.5507],
        [ 4.7115, -3.9215],
        [ 4.9390, -4.0994],
        [ 4.9544, -4.1129],
        [ 4.7737, -3.9743],
        [ 4.2837, -3.5980],
        [ 4.1222, -3.4630],
        [ 4.9971, -4.1495],
        [ 4.4445, -3.7206],
        [ 4.0653, -3.4093],
        [ 4.6636, -3.8923],
        [ 4.6489, -3.8861],
        [ 3.6769, -3.1174],
        [ 3.7471, -3.1523],
        [ 4.3140, -3.6189],
        [ 5.

Output: tensor([[ 1.9509, -1.7274],
        [ 2.4398, -2.1213],
        [ 1.9058, -1.6865],
        [ 1.9983, -1.7574],
        [ 1.8668, -1.6575],
        [ 2.6757, -2.3074],
        [ 2.2523, -1.9679],
        [ 1.8951, -1.6848],
        [ 1.7135, -1.5316],
        [ 1.7874, -1.5954],
        [ 2.0405, -1.8001],
        [ 1.8618, -1.6516],
        [ 1.9638, -1.7326],
        [ 1.6919, -1.5191],
        [ 0.8344, -0.8201],
        [ 2.1780, -1.9099],
        [ 1.9453, -1.7163],
        [ 2.2922, -1.9985],
        [ 1.7777, -1.5869],
        [ 1.8654, -1.6582],
        [ 2.0822, -1.8260],
        [ 2.0209, -1.7807],
        [ 2.0179, -1.7806],
        [ 1.6030, -1.4398],
        [ 1.9598, -1.7353],
        [ 2.4147, -2.1018],
        [ 1.7983, -1.6063],
        [ 2.3137, -2.0268],
        [ 1.4987, -1.3578],
        [ 2.0546, -1.8047],
        [ 1.9128, -1.6977],
        [ 2.3648, -2.0615],
        [ 2.1530, -1.8905],
        [ 2.2980, -2.0059],
        [ 2.1344, -1.8702],
        [ 1.

Output: tensor([[ 1.7329, -1.6160],
        [ 1.3896, -1.3385],
        [ 1.8935, -1.7623],
        [ 2.1485, -1.9707],
        [ 1.6447, -1.5528],
        [ 1.6437, -1.5504],
        [ 2.2802, -2.0624],
        [ 0.6590, -0.6871],
        [ 1.8468, -1.7015],
        [ 1.6582, -1.5588],
        [ 1.9986, -1.8410],
        [ 2.1628, -1.9879],
        [ 1.7544, -1.6362],
        [ 1.6901, -1.5982],
        [ 2.0896, -1.9345],
        [ 2.0659, -1.9109],
        [ 2.0055, -1.8494],
        [ 1.8766, -1.7498],
        [ 1.2979, -1.2589],
        [ 2.0295, -1.8748],
        [ 2.2757, -2.0776],
        [ 1.4775, -1.4137],
        [ 2.1096, -1.9400],
        [ 2.1970, -2.0108],
        [ 1.8411, -1.7171],
        [ 2.0444, -1.8842],
        [ 2.1180, -1.9433],
        [ 2.0092, -1.8474],
        [ 1.5925, -1.4895],
        [ 2.0922, -1.9295],
        [ 2.0123, -1.8555],
        [ 2.3064, -2.0868],
        [ 1.6411, -1.5344],
        [ 1.4327, -1.3673],
        [ 2.2864, -2.0829],
        [ 1.

Batch:  torch.Size([90, 20, 375])
Loss:  tensor(0.0697, grad_fn=<NllLossBackward>)
Output: tensor([[ 1.3237, -1.3039],
        [ 1.4560, -1.4259],
        [ 1.2790, -1.2543],
        [ 1.4131, -1.3783],
        [ 1.2786, -1.2730],
        [ 1.5470, -1.5264],
        [ 1.3260, -1.2903],
        [ 1.5224, -1.4690],
        [ 1.4488, -1.4277],
        [ 1.3161, -1.2976],
        [ 1.1015, -1.0975],
        [ 1.3628, -1.3389],
        [ 1.5488, -1.5115],
        [ 0.7718, -0.8004],
        [ 1.3993, -1.3655],
        [ 1.4031, -1.3768],
        [ 1.6180, -1.5758],
        [ 1.3060, -1.3010],
        [ 1.3702, -1.3450],
        [ 1.4622, -1.4365],
        [ 1.4263, -1.3919],
        [ 1.2933, -1.2710],
        [ 1.3370, -1.3056],
        [ 0.7199, -0.7699],
        [ 1.4001, -1.3702],
        [ 1.2349, -1.2276],
        [ 1.3240, -1.2997],
        [ 1.5332, -1.5168],
        [ 1.3270, -1.3035],
        [ 1.6294, -1.5876],
        [ 1.4594, -1.4367],
        [ 1.3030, -1.2826],
        [ 1.5

...

Output: tensor([[ 1.4868, -1.4764],
        [ 1.1403, -1.1715],
        [ 1.4178, -1.4123],
        [ 1.4468, -1.4357],
        [ 1.4624, -1.4579],
        [ 1.2817, -1.2929],
        [ 2.1789, -2.0888],
        [ 1.7003, -1.6752],
        [ 2.1063, -2.0221],
        [ 1.5697, -1.5478],
        [ 1.6687, -1.6338],
        [ 1.5184, -1.4997],
        [ 1.2885, -1.3044],
        [ 1.2331, -1.2535],
        [ 1.0418, -1.0814],
        [ 1.3330, -1.3368],
        [ 1.5691, -1.5446],
        [ 1.1492, -1.1760],
        [ 1.3262, -1.3383],
        [ 1.4448, -1.4329],
        [ 1.4396, -1.4291],
        [ 1.3743, -1.3820],
        [ 1.2814, -1.2911],
        [ 1.4554, -1.4434],
        [ 1.5779, -1.5559],
        [ 1.5029, -1.4857],
        [ 0.9606, -1.0102],
        [ 1.9242, -1.8639],
        [ 1.4580, -1.4462],
        [ 1.1360, -1.1626],
        [ 1.5943, -1.5670],
        [ 2.1322, -2.0471],
        [ 1.5509, -1.5346],
        [ 1.0939, -1.1284],
        [ 1.5646, -1.5428],
        [ 1.

Batch:  torch.Size([90, 20, 375])
Loss:  tensor(0.0714, grad_fn=<NllLossBackward>)
Output: tensor([[ 1.0677, -1.1080],
        [ 1.3039, -1.3147],
        [ 1.4882, -1.4797],
        [ 1.4046, -1.4069],
        [ 1.2254, -1.2516],
        [ 0.7001, -0.7872],
        [ 1.5609, -1.5494],
        [ 1.7022, -1.6705],
        [ 1.3704, -1.3742],
        [ 1.0748, -1.1143],
        [ 1.7376, -1.6978],
        [ 1.4715, -1.4636],
        [ 1.4127, -1.4111],
        [ 1.6776, -1.6526],
        [ 1.4264, -1.4249],
        [ 1.5478, -1.5320],
        [ 1.3046, -1.3162],
        [ 0.8014, -0.8770],
        [ 1.3971, -1.3998],
        [ 1.1695, -1.1978],
        [ 1.4823, -1.4722],
        [ 1.2835, -1.3018],
        [ 1.6725, -1.6417],
        [ 1.7345, -1.6931],
        [ 1.2455, -1.2628],
        [ 0.9874, -1.0399],
        [ 1.2279, -1.2475],
        [ 0.9237, -0.9841],
        [ 1.4229, -1.4261],
        [ 1.2667, -1.2824],
        [ 1.2519, -1.2693],
        [ 1.3513, -1.3556],
        [ 1.9

Output: tensor([[ 2.7129, -2.5396],
        [ 2.5677, -2.4130],
        [ 2.3502, -2.2180],
        [ 2.5536, -2.3982],
        [ 1.5406, -1.5062],
        [ 2.1064, -2.0072],
        [ 3.1301, -2.9124],
        [ 2.9605, -2.7634],
        [ 3.7526, -3.4622],
        [ 2.6602, -2.4960],
        [ 2.3144, -2.1935],
        [ 2.9153, -2.7196],
        [ 2.8586, -2.6704],
        [ 2.7942, -2.6165],
        [ 2.5227, -2.3705],
        [ 3.7000, -3.4228],
        [ 2.6112, -2.4491],
        [ 2.5777, -2.4179],
        [ 2.6231, -2.4660],
        [ 3.5866, -3.3192],
        [ 2.1966, -2.0828],
        [ 2.4801, -2.3326],
        [ 2.4806, -2.3373],
        [ 2.4555, -2.3101],
        [ 1.8922, -1.8148],
        [ 2.9657, -2.7678],
        [ 2.4887, -2.3435],
        [ 2.2191, -2.1032],
        [ 2.2338, -2.1206],
        [ 2.7161, -2.5424],
        [ 3.2673, -3.0382],
        [ 2.6599, -2.4965],
        [ 2.7375, -2.5709],
        [ 2.8405, -2.6539],
        [ 2.6944, -2.5273],
        [ 2.

Output: tensor([[ 2.0560, -1.9711],
        [ 1.7607, -1.7089],
        [ 1.8296, -1.7719],
        [ 1.6381, -1.5947],
        [ 1.2875, -1.2889],
        [ 2.0733, -1.9967],
        [ 2.3798, -2.2677],
        [ 1.6599, -1.6188],
        [ 2.1074, -2.0157],
        [ 1.3553, -1.3475],
        [ 1.9062, -1.8384],
        [ 2.3383, -2.2186],
        [ 1.6287, -1.5885],
        [ 1.4111, -1.3939],
        [ 2.0276, -1.9451],
        [ 1.4600, -1.4423],
        [ 1.8037, -1.7455],
        [ 2.2221, -2.1235],
        [ 1.7334, -1.6834],
        [ 1.7496, -1.6975],
        [ 1.6511, -1.6081],
        [ 2.0009, -1.9208],
        [ 1.7549, -1.7051],
        [ 2.5367, -2.3995],
        [ 1.4996, -1.4750],
        [ 1.7653, -1.7135],
        [ 1.7828, -1.7314],
        [ 1.4172, -1.4044],
        [ 2.3666, -2.2441],
        [ 1.8156, -1.7554],
        [ 1.8007, -1.7404],
        [ 1.7946, -1.7398],
        [ 1.8163, -1.7584],
        [ 1.7619, -1.7099],
        [ 1.9238, -1.8502],
        [ 1.

KeyboardInterrupt: 

### Baseline

In [ ]:
heldout = int(len(dataset) * 0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - heldout, heldout])

model = Baseline(add_brnn=False)
config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    
    num_epochs=200,

#     num_epochs=200,
    is_notebook=True
)

train(model, train_dataset, config)

## Testing

In [ ]:
y_true = dataset.labels[test_dataset.indices]
y_pred, test_acc = test(model, test_dataset, config)
print(len(test_dataset))
print(dataset.labels[test_dataset.indices].sum().item())

In [ ]:
results = pd.DataFrame(classification_report(y_true, y_pred, zero_division=0, output_dict=True)).transpose()
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
auc_score = metrics.auc(fpr, tpr)

In [ ]:
display(results)
print("Specificity:", specificity)
print("AUC:", auc_score)

In [ ]:
lr_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC=%.3f' % (lr_auc))
lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Baseline model')

pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:
pr_auc = average_precision_score(y_true, y_pred)
print("PR AUC:", specificity)